In [2]:

import pandas as pd

In [3]:
estrenos_por_mes = pd.read_csv("data/estrenos_por_mes.csv", sep=",")
estrenos_por_dia = pd.read_csv("data/estrenos_por_dia.csv", sep=",")
movies = pd.read_csv("data/movies_etl.csv", sep=",")
actors = pd.read_csv("data/actors.csv", sep=",")
directors = pd.read_csv("data/directors.csv", sep=",")
df = pd.read_csv("data/ml_df.csv", sep=",")
pelis = movies[['title','release_year','vote_count','vote_average']][movies['vote_count'] > 2000]


-----

# 1. cantidad_filmaciones_mes

In [4]:
def cantidad_filmaciones_mes( mes ): 
    '''
    Se ingresa un mes en idioma Español. Debe devolver la cantidad de películas que fueron estrenadas en el mes consultado en la totalidad del dataset.
    Ejemplo de retorno: X cantidad de películas fueron estrenadas en el mes de X
    '''
    mes = mes.lower().strip()

    if mes == 'setiembre':
        mes = 'septiembre'
    if mes in ['enero','febrero','marzo','abril','mayo','junio','julio','agosto','septiembre','octubre','noviembre','diciembre']:
        respuesta = estrenos_por_mes[estrenos_por_mes.mes == mes].iloc[0,1]
        respuesta = int(respuesta)

    else:
        respuesta = 'Ud. ingreso: ' + mes + '. Por favor, ingrese el nombre del mes que desea consultar en español'
    
    return {'mes':mes, 'cantidad':respuesta}

In [5]:
cantidad_filmaciones_mes('enro')


{'mes': 'enro',
 'cantidad': 'Ud. ingreso: enro. Por favor, ingrese el nombre del mes que desea consultar en español'}

----------

# 2. cantidad_filmaciones_dia

In [6]:
 # @app.get('/cantidad_filmaciones_dia/{dia}')

def cantidad_filmaciones_dia( dia ): 
    '''
    Se ingresa un día en idioma Español. Debe devolver la cantidad de películas que fueron estrenadas en día consultado en la totalidad del dataset.
    Ejemplo de retorno: X cantidad de películas fueron estrenadas en los días X
    '''
    # ver como hacer poara que solo admita strings
    dia = dia.lower().strip()

    if dia in ['miercoles','mièrcoles']:
        dia = 'miércoles'
    
    if dia in ['sabado','sàbado']:
         dia = 'sábado'
    
    if dia in ['lunes','martes','miércoles','jueves', 'viernes', 'sábado', 'domingo']:
        respuesta  = estrenos_por_dia[estrenos_por_dia.dia == dia].iloc[0,1]
        respuesta = int(respuesta)

    else:
        respuesta = 'Ud. ingreso: ' + dia + '. Por favor, ingrese el nombre del dia que desea consultar en español'
        
    return {'dia':dia, 'cantidad':respuesta}

In [7]:
cantidad_filmaciones_dia('lunes') 


{'dia': 'lunes', 'cantidad': 3499}

------

# 3. score_titulo

In [8]:
# @app.get('/score_titulo/{titulo_de_la_filmación}')

def score_titulo( titulo_de_la_filmación ): 
    '''
    Se ingresa el título de una filmación esperando como respuesta el título, el año de estreno y el score.
    Ejemplo de retorno: La película X fue estrenada en el año X con un score/popularidad de X
    '''
    pelis = (
    movies[movies['title']
    == titulo_de_la_filmación.title().strip()]
    [['title','release_year','popularity']]
    )


    titulos = pelis['title'].tolist()
    anios = pelis['release_year'].tolist()
    popularidad = pelis['popularity'].tolist()


    return {'titulo':titulos, 'anio':anios, 'popularidad':popularidad}

In [9]:
score_titulo( 'titanic' )

{'titulo': ['Titanic', 'Titanic', 'Titanic'],
 'anio': [1997, 1953, 1996],
 'popularidad': [26.88907, 12.90707, 3.435807]}

-----------

# 4. votos_titulo

In [10]:
def votos_titulo( titulo_de_la_filmación ): 
    ''' Se ingresa el título de una filmación esperando como respuesta el título, la cantidad de votos y el valor promedio de las votaciones. 
    La misma variable deberá de contar con al menos 2000 valoraciones, 
    caso contrario, debemos contar con un mensaje avisando que no cumple esta condición y que por ende, no se devuelve ningun valor.
    Ejemplo de retorno: La película X fue estrenada en el año X. La misma cuenta con un total de X valoraciones, con un promedio de X.
    '''

    pelis = movies[['title','release_year','vote_count','vote_average']][movies['vote_count'] > 2000]

    mensaje = "La pelicula {} no cuenta con las valoraciones suficientes para calcular el valor promedio de las votaciones".format(titulo_de_la_filmación)


    if pelis['title'].str.contains(titulo_de_la_filmación).any():
        pelis = pelis[pelis['title']==titulo_de_la_filmación]
        anios = pelis['release_year'].tolist()
        vote_count = pelis['vote_count'].tolist()
        vote_average = pelis['vote_average'].tolist()
  
        data = {
            'titulo':titulo_de_la_filmación, 
            'anio': anios, 
            'voto_total':vote_count, 
            'voto_promedio':vote_average}
        
    else:
        data = {mensaje}

    return data


In [11]:
print (votos_titulo( 'Titanic' )) # varias pelis, solo 1 cumple la condición
print (votos_titulo( 'The Toy' )) # 2 pelis que no cumplen la condicion
print (votos_titulo( 'Grumpier Old Men' )) # 1 peli que no cumple la condicion
print (votos_titulo('The Mummy')) # varias pelis, que cumplen la condicion

{'titulo': 'Titanic', 'anio': [1997], 'voto_total': [7770], 'voto_promedio': [7]}
{'La pelicula The Toy no cuenta con las valoraciones suficientes para calcular el valor promedio de las votaciones'}
{'La pelicula Grumpier Old Men no cuenta con las valoraciones suficientes para calcular el valor promedio de las votaciones'}
{'titulo': 'The Mummy', 'anio': [1999, 2017], 'voto_total': [2796, 2122], 'voto_promedio': [6, 5]}


----------

# 5. get_actor

In [48]:
def get_actor( nombre_actor ): 
    '''
    Se ingresa el nombre de un actor que se encuentre dentro de un dataset debiendo devolver el éxito del mismo medido a través del retorno. 
    Además, la cantidad de películas que en las que ha participado y el promedio de retorno. La definición no deberá considerar directores.
    Ejemplo de retorno: El actor X ha participado de X cantidad de filmaciones, el mismo ha conseguido un retorno de X con un promedio de X por filmación
    '''

    if (actors == nombre_actor).any().any():
    
        id_actor = actors[actors.name == nombre_actor.title().strip()].iloc[0,0]

        df = (
            pd.merge(
                actors[actors['id_actor'] == id_actor],
                movies[['id_movie','title','revenue']],
                on='id_movie', 
                how='inner')
            )
        
        cantidad = df.shape[0] 

        ganancia_total = int(df.revenue.sum())

        promedio = int(df.revenue.mean().round(2))

        data = {'actor':nombre_actor, 'cantidad_filmaciones':cantidad, 'retorno_total':ganancia_total, 'retorno_promedio':promedio}

    else:
        mensaje = 'El actor {} no se encuentra en la base de datos.'.format(nombre_actor)
        
        data = { mensaje }
    return data   




In [50]:
get_actor( 'Tom Hanks' )

{'actor': 'Tom Hanks',
 'cantidad_filmaciones': 71,
 'retorno_total': 9936142170,
 'retorno_promedio': 139945664}

In [36]:
actors[actors.name == 'Tom Hanks'.title().strip()].iloc[0,0]

31

--------

# 6. get_director

In [14]:
def get_director( nombre_director ): 
    '''
    Se ingresa el nombre de un director que se encuentre dentro de un dataset debiendo devolver el éxito del mismo medido a través del retorno. 
    Además, deberá devolver el nombre de cada película con la fecha de lanzamiento, retorno individual, costo y ganancia de la misma.
    '''
    
    id_director = directors[directors.name == nombre_director.title().strip()].iloc[0,0]

    df = (
        pd.merge(
            directors[directors['id_director'] == id_director],
            movies[['id_movie','title','release_year','revenue','budget','return']],
            on='id_movie', 
            how='inner')
    )
    
    retorno_total_director = df['return'].sum()
    peliculas = df['title'].tolist()
    anios = df['release_year'].tolist()
    retorno_peliculas = df['return'].tolist()
    budget_peliculas = df['budget'].tolist()
    revenue_peliculas = df['revenue'].tolist()

    data = {
        'director':nombre_director, 
        'retorno_total_director':retorno_total_director, 
        'peliculas':peliculas, 
        'anio':anios, 
        'retorno_pelicula':retorno_peliculas, 
        'budget_pelicula':budget_peliculas, 
        'revenue_pelicula':revenue_peliculas}

    return data
    

In [15]:
get_director('Bob Peterson')

{'director': 'Bob Peterson',
 'retorno_total_director': 1.7635100807292907,
 'peliculas': ['Finding Nemo',
  'Up',
  'Inside Out',
  'The Good Dinosaur',
  'Finding Dory',
  'Cars 3'],
 'anio': [2003, 2009, 2015, 2015, 2016, 2017],
 'retorno_pelicula': [0.0999643174178626,
  0.2380631458875907,
  0.2040551771075688,
  0.5272257144599096,
  0.1944445464468127,
  0.4997571794095461],
 'budget_pelicula': [94000000,
  175000000,
  175000000,
  175000000,
  200000000,
  175000000],
 'revenue_pelicula': [940335536,
  735099082,
  857611174,
  331926147,
  1028570889,
  350170057]}

------

In [16]:
movies['title'].str.contains('spider').any()

True